In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from IPython.display import Image, display

url = 'https://news.nate.com/recent?mid=n0100'
print(url)

req_header = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
}

res = requests.get(url, headers=req_header)
print(type(res))
print(res.status_code)

if res.ok:
    res.encoding = 'euc-kr'
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    
    tags = soup.select("div.postListType.noListTitle div.mlt01")
    print(len(tags))

    for idx,div_tag in enumerate(tags,1): 
        print(f'============>> {idx}')       
        a_tag = div_tag.find('a')
        a_join_url = urljoin(url,a_tag['href'])
        print(f'뉴스기사 링크 = {a_join_url}')

        img_tag = div_tag.select_one('span.ib img')
        if img_tag:
            photo_url = urljoin(url,img_tag['src'])
            print(photo_url)
            display(Image(url=photo_url))

        h2_tag = div_tag.select_one('span.tb h2.tit')
        title = h2_tag.text
        print(title)

else:
    print(f'에러코드 = {res.status_code}')

In [ ]:
import requests
from bs4 import BeautifulSoup
from pathlib import Path

def download_one_episode(title,no,url):
    
    req_header = {'referer': url}
    
    res = requests.get(url)
    print(res.ok)
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
               
        img_tags = soup.select("img[src*='IMAG01']")
        # list comprehension 사용
        imgurl_list = [img['src'] for img in img_tags]
        print(len(imgurl_list))
        
        save_dir = Path('img') / title / str(no)
        save_dir.mkdir(parents=True, exist_ok=True)

        for idx,img_url in enumerate(imgurl_list,1):
            res = requests.get(img_url,headers=req_header)
            res.raise_for_status()

            file_name = Path(img_url).name
            save_path = save_dir / file_name
            save_path.write_bytes(res.content)
            print(f'다운로드 완료: {save_path} ({save_path.stat().st_size:,} bytes)')
                
if __name__ == '__main__':                
    download_one_episode('낢이사는이야기',7,'https://comic.naver.com/webtoon/detail?titleId=833255&no=8')

In [ ]:
# https://comic.naver.com/api/product/list?titleId=838708 -> 회차 목록 url

In [ ]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
from urllib.parse import urlparse, parse_qs

def download_all_episode(title,episode_url):
    # url을 파싱해서 titleId를 알아내기
    parsed_url = urlparse(episode_url)
    query_params = parse_qs(parsed_url.query)
    title_id = query_params.get('titleId', [''])[0]
    
    api_url = f'https://comic.naver.com/api/article/list?titleId={title_id}'
    res = requests.get(api_url)
    print(res.status_code)    
    if res.ok:
        #pprint(res.json()['articleList'])
        for article in res.json()['articleList']:
            no = article['no']
            detail_url = f'https://comic.naver.com/webtoon/detail?titleId={title_id}&no={no}'
            print(detail_url)
        

if __name__ == '__main__': 
    download_all_episode('삽가능','https://comic.naver.com/webtoon/list?titleId=838708')

In [3]:
from urllib.parse import urlparse, parse_qs

"""url에 titleId를 반환하는 함수"""
def get_title_id(url):
    parsed_url = urlparse(url)
    query_params = parse_qs(parsed_url.query)
    title_id = query_params.get('titleId', [''])[0]
    return title_id

#테스트 하기
url = 'https://comic.naver.com/webtoon/list?titleId=826419'
print(get_title_id(url))  # 출력: 826419


826419


In [5]:
def calculate_pages(total_items, items_per_page=20):
    """총 페이지 수 계산 함수"""
    return (total_items + items_per_page - 1) // items_per_page

# 예제 사용
total_items = 49
items_per_page = 20

total_pages = calculate_pages(total_items)
print(f"총 {total_items}개의 항목을 {items_per_page}개씩 출력할 때 필요한 페이지 수: {total_pages}")
# 출력: 총 49개의 항목을 20개씩 출력할 때 필요한 페이지 수: 3

총 49개의 항목을 20개씩 출력할 때 필요한 페이지 수: 3


In [6]:
import requests

def download_all_episode(title,episode_url):
    title_id = get_title_id(episode_url)

    ajax_url = f'https://comic.naver.com/api/article/list?titleId={title_id}'               
    res = requests.get(ajax_url)

    if res.ok:
        total_count = res.json()['totalCount']
        for count in range(calculate_pages(total_count)):
            # count 변수 0,1,2 // page 번호는 1,2,3
            page = count + 1
            req_param = { "page": page }
            print(req_param)
            res = requests.get(ajax_url, params=req_param)
            for article in res.json()['articleList']:
                no = article['no']
                detail_url = f'https://comic.naver.com/webtoon/detail?titleId={title_id}&no={no}'
                print(detail_url)
        

if __name__ == '__main__': 
    download_all_episode('삽가능','https://comic.naver.com/webtoon/list?titleId=838708')

{'page': 1}
https://comic.naver.com/webtoon/detail?titleId=838708&no=42
https://comic.naver.com/webtoon/detail?titleId=838708&no=41
https://comic.naver.com/webtoon/detail?titleId=838708&no=40
https://comic.naver.com/webtoon/detail?titleId=838708&no=39
https://comic.naver.com/webtoon/detail?titleId=838708&no=38
https://comic.naver.com/webtoon/detail?titleId=838708&no=37
https://comic.naver.com/webtoon/detail?titleId=838708&no=36
https://comic.naver.com/webtoon/detail?titleId=838708&no=35
https://comic.naver.com/webtoon/detail?titleId=838708&no=34
https://comic.naver.com/webtoon/detail?titleId=838708&no=33
https://comic.naver.com/webtoon/detail?titleId=838708&no=32
https://comic.naver.com/webtoon/detail?titleId=838708&no=31
https://comic.naver.com/webtoon/detail?titleId=838708&no=30
https://comic.naver.com/webtoon/detail?titleId=838708&no=29
https://comic.naver.com/webtoon/detail?titleId=838708&no=28
https://comic.naver.com/webtoon/detail?titleId=838708&no=27
https://comic.naver.com/webt

In [7]:
import requests
from time import sleep

def download_all_episode(title,episode_url):
    title_id = get_title_id(episode_url)

    ajax_url = f'https://comic.naver.com/api/article/list?titleId={title_id}'               
    res = requests.get(ajax_url)

    if res.ok:
        total_count = res.json()['totalCount']
        for count in range(calculate_pages(total_count)):
            page = count + 1
            req_param = { "page": page}
            print(req_param)
            res = requests.get(ajax_url, params=req_param)
            for article in res.json()['articleList']:
                no = article['no']
                detail_url = f'https://comic.naver.com/webtoon/detail?titleId={title_id}&no={no}'
                print(detail_url)
                download_one_episode(title,no,detail_url)
                #0.5초간 프로세스를 중지함, 기계가 아니라 사람처럼 보이게 하려고
                sleep(0.5)

if __name__ == '__main__': 
    #download_all_episode('롤플레잉','https://comic.naver.com/webtoon/list?titleId=826419')
    download_all_episode('삽가능','https://comic.naver.com/webtoon/list?titleId=838708')

{'page': 1}
https://comic.naver.com/webtoon/detail?titleId=838708&no=42
True
205
다운로드 완료: img\삽가능\42\20260102164412_0e94a97e1e027b60f3fb88e71224bf47_IMAG01_1.jpg (264,583 bytes)
다운로드 완료: img\삽가능\42\20260102164412_0e94a97e1e027b60f3fb88e71224bf47_IMAG01_2.jpg (141,027 bytes)
다운로드 완료: img\삽가능\42\20260102164412_0e94a97e1e027b60f3fb88e71224bf47_IMAG01_3.jpg (278,437 bytes)
다운로드 완료: img\삽가능\42\20260102164412_0e94a97e1e027b60f3fb88e71224bf47_IMAG01_4.jpg (381,245 bytes)
다운로드 완료: img\삽가능\42\20260102164412_0e94a97e1e027b60f3fb88e71224bf47_IMAG01_5.jpg (305,509 bytes)
다운로드 완료: img\삽가능\42\20260102164412_0e94a97e1e027b60f3fb88e71224bf47_IMAG01_6.jpg (270,715 bytes)
다운로드 완료: img\삽가능\42\20260102164412_0e94a97e1e027b60f3fb88e71224bf47_IMAG01_7.jpg (151,878 bytes)
다운로드 완료: img\삽가능\42\20260102164412_0e94a97e1e027b60f3fb88e71224bf47_IMAG01_8.jpg (200,775 bytes)
다운로드 완료: img\삽가능\42\20260102164412_0e94a97e1e027b60f3fb88e71224bf47_IMAG01_9.jpg (255,340 bytes)
다운로드 완료: img\삽가능\42\20260102164412_0e94a97e1e0